In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df=pd.read_csv("train.csv")

# New Section

In [6]:
df["lead_time"] =np.log(df["lead_time"])
df["agent"] =np.log(df["agent"])
df["adr"] = np.log(df["adr"])



/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [0]:
df=df.drop(['id'], axis=1)
df=df.drop(['arrival_date'], axis=1)
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
for i in df.columns:
  if df[i].dtypes =="object":
    df[i]=le.transform(df[i])
    le.fit(df[i])
    
    

In [0]:
Y=df["is_canceled"]
X=df.drop(["is_canceled"],axis=1)
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [17]:
grid = {
    'learning_rate': [0.03, 0.1],
    'depth':[4, 6, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}
import catboost
cat_model = catboost.CatBoostClassifier(
    iterations=1000)
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
random_search=RandomizedSearchCV(cat_model,param_distributions=grid,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
random_search.fit(X,Y)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  9.6min finished


0:	learn: 0.6146708	total: 145ms	remaining: 2m 24s
1:	learn: 0.5555270	total: 205ms	remaining: 1m 42s
2:	learn: 0.5083255	total: 273ms	remaining: 1m 30s
3:	learn: 0.4758565	total: 339ms	remaining: 1m 24s
4:	learn: 0.4517645	total: 407ms	remaining: 1m 21s
5:	learn: 0.4291431	total: 467ms	remaining: 1m 17s
6:	learn: 0.4130672	total: 532ms	remaining: 1m 15s
7:	learn: 0.4018277	total: 593ms	remaining: 1m 13s
8:	learn: 0.3898988	total: 659ms	remaining: 1m 12s
9:	learn: 0.3796439	total: 722ms	remaining: 1m 11s
10:	learn: 0.3723042	total: 783ms	remaining: 1m 10s
11:	learn: 0.3641491	total: 845ms	remaining: 1m 9s
12:	learn: 0.3585820	total: 911ms	remaining: 1m 9s
13:	learn: 0.3539494	total: 975ms	remaining: 1m 8s
14:	learn: 0.3502639	total: 1.03s	remaining: 1m 8s
15:	learn: 0.3448239	total: 1.1s	remaining: 1m 7s
16:	learn: 0.3419165	total: 1.16s	remaining: 1m 7s
17:	learn: 0.3378068	total: 1.22s	remaining: 1m 6s
18:	learn: 0.3344630	total: 1.29s	remaining: 1m 6s
19:	learn: 0.3319030	total: 1.3

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=<catboost.core.CatBoostClassifier object at 0x7f7c0e0d2cf8>,
                   iid='deprecated', n_iter=5, n_jobs=-1,
                   param_distributions={'depth': [4, 6, 10],
                                        'l2_leaf_reg': [1, 3, 5, 7, 9],
                                        'learning_rate': [0.03, 0.1]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=3)

In [18]:
random_search.best_estimator_


In [19]:
random_search.best_params_

{'depth': 10, 'l2_leaf_reg': 5, 'learning_rate': 0.1}

In [0]:
model_final = catboost.CatBoostClassifier(iterations=600
                                          ,learning_rate=0.1
                                          ,l2_leaf_reg=5
                                          ,depth=10
                                          ,rsm=0.98
                                          ,loss_function= 'Logloss'
                                           ,eval_metric='AUC'
                                           ,use_best_model=True
                                          ,random_seed=42)

In [32]:
model_final.fit(X_train,y_train,eval_set=(X_test,y_test))

0:	test: 0.8859134	best: 0.8859134 (0)	total: 53.7ms	remaining: 32.2s
1:	test: 0.8982683	best: 0.8982683 (1)	total: 108ms	remaining: 32.4s
2:	test: 0.9042758	best: 0.9042758 (2)	total: 163ms	remaining: 32.4s
3:	test: 0.9033977	best: 0.9042758 (2)	total: 215ms	remaining: 32s
4:	test: 0.9052244	best: 0.9052244 (4)	total: 273ms	remaining: 32.5s
5:	test: 0.9061746	best: 0.9061746 (5)	total: 327ms	remaining: 32.4s
6:	test: 0.9105019	best: 0.9105019 (6)	total: 381ms	remaining: 32.3s
7:	test: 0.9129219	best: 0.9129219 (7)	total: 446ms	remaining: 33s
8:	test: 0.9141978	best: 0.9141978 (8)	total: 516ms	remaining: 33.9s
9:	test: 0.9153554	best: 0.9153554 (9)	total: 567ms	remaining: 33.5s
10:	test: 0.9176506	best: 0.9176506 (10)	total: 620ms	remaining: 33.2s
11:	test: 0.9187816	best: 0.9187816 (11)	total: 674ms	remaining: 33s
12:	test: 0.9191519	best: 0.9191519 (12)	total: 729ms	remaining: 32.9s
13:	test: 0.9193259	best: 0.9193259 (13)	total: 781ms	remaining: 32.7s
14:	test: 0.9203082	best: 0.920

In [33]:
from sklearn.metrics import accuracy_score
# make predictions for test data
y_pred = model_final.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 87.80%


In [0]:
tst=pd.read_csv("test.csv")
id_val=tst['id']
tst=tst.drop(['id'], axis=1)
tst=tst.drop(['arrival_date'], axis=1)

from sklearn import preprocessing
le=preprocessing.LabelEncoder()

for i in tst.columns:
    if tst[i].dtypes =="object":
        le.fit(tst[i])
        tst[i]=le.transform(tst[i])
        

In [0]:
test_pred=model_final.predict(tst)

In [36]:
test_pred

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
submit_xg =pd.DataFrame()
submit_xg["id"]=id_val
submit_xg["is_canceled"] =test_pred

In [0]:
submit_xg.to_csv("submit.csv")

In [0]:
from google.colab import files
files.download("submit.csv")